# trajectory_buoy_erddap

compare wave buoy path data with what parcels says

compare the path of a buoy with a particle simulation on erddap data (NOT thredds)

data downloaded/queried from here: https://erddap.sccoos.org/erddap/griddap/roms_ncst.html

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import numpy as np
from parcels import FieldSet
import xarray as xr

import utils
from parcels_utils import buoycsv_to_particleds, clean_erddap_ds, HFRGrid
from parcels_sim import simulation, generate_sim_gif
from plot_utils import plot_trajectories, plot_points_fieldset

In [ ]:
# roms_ncst_ae30_8a12_e3ca.nc (don't use lol)
# buoy 689-02: corrected maybe? roms_ncst_540f_9ebe_9c54.nc
# buoy 704-02: roms_ncst_524b_bd8b_75c1.nc
erddap_path = utils.CURRENT_NETCDF_DIR / "roms_ncst_524b_bd8b_75c1.nc"
dataset = xr.open_dataset(erddap_path)
dataset = clean_erddap_ds(dataset)
dataset

In [ ]:
grid = HFRGrid(dataset)

In [ ]:
# 5-01 west coast (2009 February, too old for thredds)
# 590-09 west coast (2018 October)
# 704-02 west coast
# 689-02 west coast
# 612-08 east coast
# 731-02 east coast 
buoy_id = "704-02"
csv_path = utils.WAVEBUOY_DATA_DIR / f"wavebuoy_{buoy_id}.csv"
buoy_ds = buoycsv_to_particleds(csv_path)
# buoy_ds = buoy_ds.where(buoy_ds["time"] >= dataset["time"].min(), drop=True)
buoy_ds

In [ ]:
filename = os.path.basename(csv_path)
# hardcoded for 704-02 specifically (index [95, 145))
buoy_ds_selected = buoy_ds.sel(obs=slice(95, 145))
nc_path_selected = utils.WAVEBUOY_DATA_DIR / f"{os.path.splitext(filename)[0]}_selected.nc"
buoy_ds_selected.to_netcdf(nc_path_selected)
buoy_ds_selected

In [ ]:
plot_points_fieldset(buoy_ds_selected["lat"].values[0], buoy_ds_selected["lon"].values[0], 0, grid, domain=grid.get_domain(), line=True)

In [ ]:
domain = grid.get_domain()
domain

In [ ]:
# domain["S"] = 32.25
# domain["N"] = 33
# domain["E"] = -117.4

domain["W"] = -117.85
domain["E"] = -117.65
domain["S"] = 32.35
domain["N"] = 32.9

In [ ]:
buoy_timestamps = buoy_ds_selected["time"].values[0].astype("datetime64[s]")
start = buoy_timestamps[0] - np.timedelta64(18, "h")
end = buoy_timestamps[-1]
start_pos = (buoy_ds_selected["lat"].isel(traj=0,obs=0).values.item(), buoy_ds_selected["lon"].isel(traj=0,obs=0).values.item())
start, end, start_pos

In [ ]:
parcels_cfg = {
    "time_range": [start, end],
    "repeat_dt": -1,
    "particles_per_dt": -1,
    "max_variation": 0.0,
    "spawn_points": [
        start_pos
    ],
    "random_spawn": True,
    "simulation_dt": 300,
    "snapshot_interval": 7200,
    "save_snapshots": True,
    "shown_domain": domain,
    "part_size": 16
}

In [ ]:
pfile_path, snap_path = simulation("erddap_buoy", grid, parcels_cfg)

In [ ]:
generate_sim_gif(
    snap_path,
    utils.PICUTRE_DIR / "erddap_buoy.gif",
    25
)

In [ ]:
plot_path = utils.create_path(utils.PICUTRE_DIR / "buoy")
plot_trajectories([nc_path_selected, pfile_path], [buoy_id, "Simulated path"], legend=True, titlestr=f"ERDDAP Comparison from {np.datetime64(start, 's')} to {np.datetime64(end, 's')}",savefile=plot_path / f"buoy_comp_erddap_{buoy_id}.png")